In [5]:
import requests
from bs4 import BeautifulSoup

In [11]:
HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en-IN;q=0.9,en;q=0.8,hi-IN;q=0.7,hi;q=0.6,mr-IN;q=0.5,mr;q=0.4",
    "Priority": "u=0, i",
    "Sec-Ch-Ua": "\"Not;A=Brand\";v=\"99\", \"Google Chrome\";v=\"139\", \"Chromium\";v=\"139\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
}

def scrape_amazon_product(url):
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"❌ Failed to fetch page, status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")
    data = {}
    # Product Title
    title_el = soup.find("span", id="productTitle")
    data['Title'] = title_el.get_text(strip=True) if title_el else None
    # Price
    price_el = (
        soup.find("span", id="priceblock_ourprice") or
        soup.find("span", id="priceblock_dealprice") or
        soup.find("span", id="price_inside_buybox") or
        soup.find("span", class_="a-price-whole") or
        soup.find("span", class_="a-offscreen")
    )
    data['Price'] = price_el.get_text(strip=True) if price_el else None
    # Rating
    rating_el = soup.find("span", class_="a-icon-alt")
    data['Rating'] = rating_el.get_text(strip=True) if rating_el else None

    # Number of reviews
    reviews_el = soup.find("span", id="acrCustomerReviewText")
    data['Reviews'] = reviews_el.get_text(strip=True) if reviews_el else None

    # Key features (bullet points)
    features = []
    feature_list = soup.find("div", id="feature-bullets")
    if feature_list:
        for li in feature_list.find_all("span", class_="a-list-item"):
            text = li.get_text(strip=True)
            if text:
                features.append(text)
    data['Features'] = features

    return data


# Example usage
if __name__ == "__main__":
    URL = "https://www.amazon.in/Samsung-Galaxy-Smartphone-Titanium-Storage/dp/B0CS5XW6TN"
    product_info = scrape_amazon_product(URL)

    if product_info:
        print("\n✅ Scraped Product Info:\n")
        for key, value in product_info.items():
            if isinstance(value, list):
                print(f"{key}:")
                for v in value:
                    print(f"  - {v}")
            else:
                print(f"{key}: {value}")



✅ Scraped Product Info:

Title: Samsung Galaxy S24 Ultra 5G AI Smartphone with Galaxy AI (Titanium Gray, 12GB, 256GB Storage), Snapdragon 8 Gen 3, 200 MP Camera with ProVisual Engine and 5000mAh Battery
Price: 97,999.
Rating: 4.4 out of 5 stars
Reviews: 2,912 ratings
Features:
  - Galaxy AI - Welcome to the era of mobile AI. With Galaxy S24 Ultra and One UI in your hands, you can unleash completely new levels of creativity, productivity and possibility.
  - Titanium Frame - Meet Galaxy S24 Ultra, the ultimate form of Galaxy Ultra built with a new titanium exterior and a 17.25cm flat display. It is an absolute marvel of design.
  - Epic Camera - With the most megapixels on a Galaxy smartphone ever (200MP) and AI processing, Galaxy S24 Ultra sets the industry standard for image quality every time you hit the shutter. Also, the new ProVisual engine recognizes objects, improves color tone, reduces noise and brings out every detail.
  - Powerful Processor - Victory can be yours with the ne